In [ ]:
#轉換成圖片

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Flatten, Softmax, Dense, Add, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw
from tensorflow.keras.applications import EfficientNetB0
import os


# 读取数据(要轉換的訓練資料)
df = open('D:/tf29/Go_competition/AIcupTutorial-main/Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',', 2)[-1] for i in df]
game_styles = [int(i.split(',', 2)[-2]) for i in df]
chars = 'abcdefghijklmnopqrs'
coordinates = {k: v for v, k in enumerate(chars)}

# 计算棋盘上点的像素位置
def get_pixel_position(move, size=224, grid_size=19):
    pixel_per_unit = (size - 20) / (grid_size - 1)
    x = 10 + pixel_per_unit * coordinates[move[1]]
    y = 10 + pixel_per_unit * coordinates[move[0]]
    return int(x), int(y)

# 准备带有棋子的棋盘图像
def prepare_board_with_pieces(moves, size=224):
    board_img = Image.new('RGB', (size, size), 'orange')
    draw = ImageDraw.Draw(board_img)

    # 确定每个格子的像素大小
    pixel_per_unit = (size - 20) / 18

    # 绘制棋盘线
    for i in range(19):
        draw.line((10, 10 + i * pixel_per_unit, size - 10, 10 + i * pixel_per_unit), fill='gray')
        draw.line((10 + i * pixel_per_unit, 10, 10 + i * pixel_per_unit, size - 10), fill='gray')

    # 绘制棋子
    for idx, move in enumerate(moves):
        color, pos = move[0], move[2:4]
        x, y = get_pixel_position(pos, size, 19)
        piece_color = 'black' if color == 'B' else 'white'
        radius = 4  # 棋子半径大小

        # 为最后一步棋绘制方形
        if idx == len(moves) - 1:
            draw.rectangle([(x - radius, y - radius), (x + radius, y + radius)], fill=piece_color)
        elif idx == len(moves) - 2:
            # 为对方最后一步棋绘制菱形
            draw.regular_polygon(((x, y), radius), 4, rotation=45, fill=piece_color)
        else:
            # 绘制圆形棋子
            draw.ellipse([(x - radius, y - radius), (x + radius, y + radius)], fill=piece_color)

    return np.array(board_img) / 255.0

# 创建保存图像的目录資料夾
save_dir = 'D:/tf29/Go_competition/AIcupTutorial-main/Training Dataset/play_style_image_v3.3'
os.makedirs(save_dir, exist_ok=True)

# 遍历数据并保存图像
for idx, (game, style) in enumerate(zip(games, game_styles)):
    moves = game.split(',')
    board_img = prepare_board_with_pieces(moves)  # 生成棋盘图像
    # 构建文件名，使用格式化确保序号为固定长度的字符串
    filename = f'PSL{idx + 1:012}_style_{style}.jpg'  # 例如 'PSL0000000001_style_1.jpg'
    save_path = os.path.join(save_dir, filename)
    Image.fromarray((board_img * 255).astype(np.uint8)).save(save_path)

print("所有图像已保存至", save_dir)


所有图像已保存至 D:/tf29/Go_competition/AIcupTutorial-main/Training Dataset/play_style_image_v3.3


In [ ]:
#將圖片分類(因為ImageDataGenerator需要

In [3]:
import os
import shutil
import numpy as np

# 设置圖片來源資料夾和目标資料夾
source_dir = 'D:/tf29/Go_competition/AIcupTutorial-main/Training Dataset/play_style_image_v3.3'
train_dir = os.path.join(source_dir, 'train')
val_dir = os.path.join(source_dir, 'val')

# 定义棋风类别
styles = ['style_1', 'style_2', 'style_3']

# 分割比例
split_ratio = 0.9

# 为每种棋风在训练集和验证集中创建子資料夾
for style in styles:
    os.makedirs(os.path.join(train_dir, style), exist_ok=True)
    os.makedirs(os.path.join(val_dir, style), exist_ok=True)

    # 获取每种棋风的文件列表
    files = [file for file in os.listdir(source_dir) if file.endswith('.jpg') and style in file]

    # 随机打乱文件列表
    np.random.shuffle(files)

    # 确定分割点
    split = int(len(files) * split_ratio)

    # 分配文件到训练集和验证集的相应子資料夾
    for file in files[:split]:
        shutil.move(os.path.join(source_dir, file), os.path.join(train_dir, style, file))
    for file in files[split:]:
        shutil.move(os.path.join(source_dir, file), os.path.join(val_dir, style, file))

print("数据分割完成。")

数据分割完成。
